In [1]:
import numpy as np
import pandas as pd

In [2]:
data_dir = '../../data/clean/'
dfTau    = pd.read_csv(data_dir + 'tau_2021.csv').sort_values(by=['short_names'])
dfepsN   = pd.read_csv(data_dir + 'epsN_2021.csv', index_col=0).sort_index(axis=1)
dfepsN_weights = dfepsN.rename_axis('short_names').reset_index().melt(id_vars='short_names')

First, we load the $\varepsilon^f_N$ weights that we use to impute the occupation-level labor market variables.

In [3]:
dfepsN_weights.groupby('short_names')['value'].sum()

short_names
accom       1.0
const       1.0
dur         1.0
edhealth    1.0
fin         1.0
gov         1.0
info        1.0
mining      1.0
nondur      1.0
other       1.0
profserv    1.0
trade       1.0
trans       1.0
Name: value, dtype: float64

We first impute the number of vacancies and unemployment in each occupation using the following:
\begin{align*}
    V_o &= \sum_i \frac{\varepsilon^f_{N_{io}}}{\varepsilon^f_{N_i}} V_{i}, \\
    H_o &= \sum_i \frac{\varepsilon^f_{N_{io}}}{\varepsilon^f_{N_i}} H_{i} ,
\end{align*}

In [4]:
uvh= pd.read_csv(data_dir + 'uvh_updated.csv')
uvh = uvh.sort_values(by=['Date', 'short_names']).dropna()
uvh = uvh.dropna(axis=0)
uvhOccu = pd.merge(uvh, dfepsN_weights, on=['short_names'])
uvhOccu.loc[:, "Unemployment"] = uvhOccu['value'] * uvhOccu['Unemployment']
uvhOccu.loc[:, "Vacancy"] = uvhOccu['value'] * uvhOccu['Vacancy']
uvhOccu.loc[:, "Hires"] = uvhOccu['value'] * uvhOccu['Hires']
uvhOccu = uvhOccu.groupby(['variable', 'Date']).aggregate({'Unemployment': 'mean', 'Vacancy': 'mean', 'Hires': 'mean'})
uvhOccu.loc[:, 'Tightness'] = uvhOccu['Vacancy'] / uvhOccu['Unemployment']
uvhOccu = uvhOccu.reset_index()
uvhOccu.to_csv("../../data/clean/uvh_updated_occu.csv", index=False)
uvhOccu

,variable,Date,Unemployment,Vacancy,Hires,Tightness
0,Admin,2000-12-01,30.896366,36.326155,37.414298,1.175742
1,Admin,2001-01-01,39.628867,37.298644,39.490191,0.941199
2,Admin,2001-02-01,38.911774,36.497210,36.681547,0.937948
3,Admin,2001-03-01,38.348697,33.634549,37.723000,0.877071
4,Admin,2001-04-01,36.073837,32.788029,35.418972,0.908914
...,...,...,...,...,...,...
5869,Trans,2022-10-01,28.287768,51.916208,36.816537,1.835288
5870,Trans,2022-11-01,31.782490,51.860161,37.502583,1.631721
5871,Trans,2022-12-01,30.900005,57.139961,37.425337,1.849189
5872,Trans,2023-01-01,35.929200,60.219499,37.976727,1.676060


In [5]:
uvhOccu.loc[:, 'Year'] = pd.to_datetime(uvhOccu['Date']).apply(lambda x: x.year)
uvhOccu_annual = uvhOccu.dropna().groupby(['variable', 'Year']).aggregate({'Unemployment': "mean", 'Vacancy': "mean", 'Hires':"mean"}).reset_index()
uvhOccu_annual.loc[:, 'Tightness'] = uvhOccu_annual['Vacancy'] / uvhOccu_annual['Unemployment']
uvhOccu_annual.to_csv('../../data/clean/uvh_annual_updated_occu.csv', index=False)
uvhOccu_annual.head()

,variable,Year,Unemployment,Vacancy,Hires,Tightness
0,Admin,2000,30.896366,36.326155,37.414298,1.175742
1,Admin,2001,40.814844,31.282181,35.865932,0.766441
2,Admin,2002,50.989995,25.115374,33.559805,0.492555
3,Admin,2003,52.920450,23.680200,32.679778,0.447468
4,Admin,2004,48.502739,26.455363,34.963454,0.545441


We then estimate the matching parameters according to:

\begin{align*}
    \log H_{o,t} = \log \phi_o  + \eta_o \log U_{o,t} + (1-\eta_o) \log V_{o,t} + \epsilon_{o,t}  
\end{align*}

In [6]:
def matching_estimation2(df):
    log_H = np.log(np.array(df[['Hires']]))
    log_U = np.log(np.array(df[['Unemployment']]))
    log_U = log_U[~np.isnan(log_H)]
    log_V = np.log(np.array(df[['Vacancy']]))
    log_V = log_V[~np.isnan(log_H)]
    log_H = log_H[~np.isnan(log_H)]
    Y = np.ones((log_H.shape[0],1))
    Y[:,0] = log_H - log_V
    X = np.ones((log_H.shape[0],2))
    X[:,1] = log_U - log_V
    theta = np.linalg.inv(X.T @ X) @ (X.T @ Y)
    theta[0] = np.exp(theta[0])
    return theta.flatten()

matching_param2 = uvhOccu.groupby("variable").apply(matching_estimation2) 
matching_efficiency = []
unemployment_elasticity = []
for i in range(matching_param2.size):
    matching_efficiency.append(matching_param2.iloc[i][0])
    unemployment_elasticity.append(matching_param2.iloc[i][1])
df_matching_param = pd.DataFrame({'occupation':matching_param2.index.values,'matching_efficiency':matching_efficiency,
                                   'unemployment_elasticity':unemployment_elasticity})
df_matching_param.to_csv('../../data/clean/matching_param_estimates_occu.csv', index=False)
df_matching_param.head()

,occupation,matching_efficiency,unemployment_elasticity
0,Admin,0.882588,0.368118
1,Agg,0.906748,0.389782
2,Arts,0.939268,0.351735
3,Bus Ops,0.878515,0.355697
4,Care,0.978076,0.377744


In [7]:
print(df_matching_param.set_index('occupation').round(3).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  matching\_efficiency &  unemployment\_elasticity \\
occupation &                      &                          \\
\midrule
Admin      &                0.883 &                    0.368 \\
Agg        &                0.907 &                    0.390 \\
Arts       &                0.939 &                    0.352 \\
Bus Ops    &                0.879 &                    0.356 \\
Care       &                0.978 &                    0.378 \\
Clean      &                1.009 &                    0.372 \\
Cons       &                1.054 &                    0.461 \\
Educ       &                0.713 &                    0.338 \\
Eng        &                0.871 &                    0.357 \\
Food S     &                1.163 &                    0.398 \\
Health P   &                0.749 &                    0.346 \\
Health S   &                0.731 &                    0.343 \\
Legal      &                0.923 &                    0.354 \\
Manag  

We compute occupation-level recruiter-producer ratio by summing the number of recruiters designated to each occupation in each sector and dividing that by the number of workers in each occupation. We impute the number of recruiters for occupation o by:
\begin{align*}
    R_o &= \sum_i \frac{\varepsilon^f_{N_{io}}}{\varepsilon^f_{N_i}} R_{i}, \\
\end{align*}
where $R_{i}$ is the number of recruiters in sector $i$. This is also implicitly assuming that the recruiting cost for the occupations are the same. 

In [8]:
dfOccuTau = pd.merge(dfTau, dfepsN_weights, on=['short_names']) 
dfOccuTau.loc[:, "TOT_EMP_HR"] = dfOccuTau['value'] * dfOccuTau['TOT_EMP_HR']
dfOccuTau = dfOccuTau.groupby('variable').aggregate({'TOT_EMP_HR': 'sum'}).reset_index()
dfOccuTau.head()

,variable,TOT_EMP_HR
0,Admin,104065.650761
1,Agg,735.273165
2,Arts,19461.245237
3,Bus Ops,125385.402737
4,Care,14811.692624


In [9]:
occ_by_ind = pd.read_excel("../../data/raw/OES/natsector_M2021_dl.xlsx")
occ_by_ind = occ_by_ind[occ_by_ind['O_GROUP'] == 'major'][['NAICS', 'NAICS_TITLE', "OCC_CODE", "OCC_TITLE", "TOT_EMP", "A_MEAN"]]
occ_by_ind.loc[:, 'TOT_EMP'] = pd.to_numeric(occ_by_ind['TOT_EMP'], errors='coerce').fillna(0)
occ_map = {'Architecture and Engineering Occupations': 'Eng', 
           'Arts, Design, Entertainment, Sports, and Media Occupations': 'Arts',
           'Building and Grounds Cleaning and Maintenance Occupations': 'Clean',
           'Business and Financial Operations Occupations': 'Bus Ops',
           'Community and Social Service Occupations': 'Soc S',
           'Computer and Mathematical Occupations': 'Math',
           'Construction and Extraction Occupations': 'Cons',
           'Educational Instruction and Library Occupations': 'Educ',
           'Farming, Fishing, and Forestry Occupations': 'Agg',
           'Food Preparation and Serving Related Occupations': 'Food S',
           'Healthcare Practitioners and Technical Occupations': 'Health P',
           'Healthcare Support Occupations': 'Health S', 
           'Installation, Maintenance, and Repair Occupations': 'Repair', 
           'Legal Occupations': 'Legal', 'Life, Physical, and Social Science Occupations':'Science',
           'Management Occupations': 'Manag', 'Office and Administrative Support Occupations': 'Admin',
           'Personal Care and Service Occupations': 'Care', 'Production Occupations': 'Prod',
           'Protective Service Occupations': 'Prot S', 'Sales and Related Occupations': 'Sales', 
           'Transportation and Material Moving Occupations': 'Trans'}
occ_emp = occ_by_ind.groupby('OCC_TITLE')['TOT_EMP'].sum().rename(index=occ_map).reset_index()
occ_emp.head()

,OCC_TITLE,TOT_EMP
0,Eng,2407010.0
1,Arts,1808320.0
2,Clean,4108790.0
3,Bus Ops,8911520.0
4,Soc S,2213830.0


In [10]:
dfOccuTau = pd.merge(dfOccuTau, occ_emp, left_on='variable', right_on='OCC_TITLE').drop(columns='OCC_TITLE')
dfOccuTau.loc[:, 'Workers'] = dfOccuTau['TOT_EMP'] - dfOccuTau['TOT_EMP_HR']
dfOccuTau.loc[:, 'Tau'] = dfOccuTau['TOT_EMP_HR'] / dfOccuTau['Workers']
dfOccuTau.to_csv('../../data/clean/tau_2021_occu.csv', index=False)
dfOccuTau.head()

,variable,TOT_EMP_HR,TOT_EMP,Workers,Tau
0,Admin,104065.650761,18299400.0,1.819533e+07,0.005719
1,Agg,735.273165,447120.0,4.463847e+05,0.001647
2,Arts,19461.245237,1808320.0,1.788859e+06,0.010879
3,Bus Ops,125385.402737,8911520.0,8.786135e+06,0.014271
4,Care,14811.692624,2566440.0,2.551628e+06,0.005805


In [11]:
print(dfOccuTau[['variable', 'Tau']].set_index('variable').round(3).to_latex())

\begin{tabular}{lr}
\toprule
{} &    Tau \\
variable &        \\
\midrule
Admin    &  0.006 \\
Agg      &  0.002 \\
Arts     &  0.011 \\
Bus Ops  &  0.014 \\
Care     &  0.006 \\
Clean    &  0.006 \\
Cons     &  0.004 \\
Educ     &  0.005 \\
Eng      &  0.017 \\
Food S   &  0.002 \\
Health P &  0.009 \\
Health S &  0.003 \\
Legal    &  0.027 \\
Manag    &  0.019 \\
Math     &  0.020 \\
Prod     &  0.006 \\
Prot S   &  0.009 \\
Repair   &  0.007 \\
Sales    &  0.005 \\
Science  &  0.014 \\
Soc S    &  0.006 \\
Trans    &  0.003 \\
\bottomrule
\end{tabular}

